In [1]:
import sys
# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")

In [2]:
import numpy as np
import os
import six
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from PIL import Image

import pandas as pd
import pickle

from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util
from object_detection.utils import dataset_util

In [3]:
with open('train.p', 'rb') as f:
    train = pickle.load(f)
print(train.shape)

with open('test.p', 'rb') as f:
    test = pickle.load(f)
print(test.shape)

test.head()

(287, 9)
(123, 9)


,filename,xmins,xmaxs,ymins,ymaxs,classes,classes_text,height,width
85,sim_1507476955.png,0.043750,0.065000,0.858333,0.931667,2,b'green',600,800
127,sim_1507476785.png,0.718750,0.736250,0.956667,0.996667,2,b'green',600,800
384,site_1507478056.602447032.png,0.305223,0.357118,0.318458,0.481637,2,b'green',1096,1368
21,sim_1507476502.png,0.405000,0.458750,0.641667,0.795000,0,b'red',600,800
101,sim_1507476952.png,0.211250,0.232500,0.896667,0.960000,2,b'green',600,800


In [4]:
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

def encode_image_array_as_jpg_str(image):
    """Encodes a numpy array into a JPEG string.
    Args:
    image: a numpy array with shape [height, width, 3].
    Returns:
    JPEG encoded image string.
    """
    image_pil = Image.fromarray(np.uint8(image))
    output = six.BytesIO()
    image_pil.save(output, format='JPEG')
    jpg_string = output.getvalue()
    output.close()
    return jpg_string

def create_record(image, row):
    
    height = row['height']
    width = row['width']
    filename = row['filename'].encode()
    encoded_image_data = encode_image_array_as_jpg_str(image)
    image_format = b'jpeg'
    xmins = [ row['xmins'] ] 
    xmaxs = [ row['xmaxs'] ]
    ymins = [ row['ymins'] ]
    ymaxs = [ row['ymaxs'] ]
    classes_text = [ row['classes_text'] ] 
    classes = [ row['classes'] ]

    tf_record = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_image_data),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_record

In [5]:
PATH_TO_TEST_IMAGES_DIR = 'finishline_images'

In [45]:
pickleFile = 'train.p'
    
writer = tf.python_io.TFRecordWriter(b'train.record')

with open(pickleFile, 'rb') as f:
    df = pickle.load(f)

df = df.dropna() # NaNs

for i in range(df.shape[0]):

    row = df.iloc[i, ]
    image = Image.open(os.path.join(PATH_TO_TEST_IMAGES_DIR, row['filename']))
    image_np = load_image_into_numpy_array(image)

    record = create_record(image_np, row)
    writer.write(record.SerializeToString())

writer.close()

In [46]:
pickleFile = 'test.p'
    
writer = tf.python_io.TFRecordWriter(b'test.record')

with open(pickleFile, 'rb') as f:
    df = pickle.load(f)

df = df.dropna() # NaNs

for i in range(df.shape[0]):

    row = df.iloc[i, ]
    image = Image.open(os.path.join(PATH_TO_TEST_IMAGES_DIR, row['filename']))
    image_np = load_image_into_numpy_array(image)

    record = create_record(image_np, row)
    writer.write(record.SerializeToString())

writer.close()